# Remote Sensing with Python

>"...the time is now right and urgent to apply space technology towards the solution of many pressing natural resources problems being compounded by population and industrial growth."

*Stewart Udall, Secretary of the Interior, September 21, 1966*

## Workflow

We will do the following:

- Use Google Earth Engine's Python API
- define an AOI (area of interest) in the Campania region
- import multiple Sentinel-2 images from the dates before and after the fires
- determine which images are best for analysis
- create various NDVI map outputs to assess the amount of fire damage

We will use Google Earth Engine's **Harmonized Sentinel-2 MSI**: MultiSpectral Instrument, Level-2A [[EE Sentinel 2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED)]

In [ ]:
from pathlib import Path
from datetime import datetime
from PIL import Image as PILImage
from io import BytesIO
import requests
import pytz

import ee
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
from IPython.display import Image

from utils import add_ee_layer

IMAGES_PATH = Path(Path.cwd().parent, "images")

In [ ]:
# Trigger the authentication flow
ee.Authenticate()

# Initialize the GEE library
ee.Initialize()

## Define filters

In [ ]:
# coordinates of Mount Vesuvius (Naples, Italy)
lat = 40.821330
lon = 14.426102

# point of interest as an ee.Geometry
poi = ee.Geometry.Point(lon,lat)

# time filters
dates = [
    ('2017-05-10', '2017-05-25'),
    ('2017-08-01', '2017-08-18'),
    ('2023-07-10', '2023-07-18'),
#     ('2024-02-18', '2024-02-25'),
]

filters = [ee.Filter.date(start, end) for start, end in dates]
comb_filter = ee.Filter.Or(*filters)

## Get Sentinel-2 data

In [ ]:
sentinel = ee.ImageCollection(
    "COPERNICUS/S2_SR_HARMONIZED"
).filterBounds(
    poi
).filter(
    comb_filter
)

In [ ]:
sentinel_list = sentinel.toList(sentinel.size())

In [ ]:
print(f'Number of images downloaded: {sentinel.size().getInfo()}')

In [ ]:
# sentinel.first().getInfo()

In [ ]:
# Bands
# sentinel.first().bandNames().getInfo()

## Display satellite images

In [ ]:
# set some parameters for the images
parameters = {
    'min': 0,
    'max': 3_000,
    'dimensions': 800,
    'bands': ['B4', 'B3', 'B2'], # bands to display (r,g,b)
}

In [ ]:
data = []
for i in range(sentinel.size().getInfo()):

    # when was this image taken?
    unix_time = ee.Image(sentinel_list.get(i)).get('GENERATION_TIME').getInfo() / 1_000
    date_time = datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d %H:%M:%S')
    
    # cloud cover
    cloud = ee.Image(sentinel_list.get(i)).get('CLOUD_COVERAGE_ASSESSMENT').getInfo()
    print(f'Image #{i} | {date_time} | cloud coverage: {np.round(cloud, 1)}%')
    
    display(
        Image(
            url=ee.Image(sentinel_list.get(i)).getThumbUrl(parameters)
        )
    )

    this_data = [i, date_time, cloud]
    data.append(this_data)    

df = pd.DataFrame(data, columns = ['Image #', 'Datetime', 'Cloud Cover'])

In [ ]:
df

## Selecting images, zooming in
Now that we have inspected our collection of images, we can pick and choose which ones are relevant for our study. Ideally, we want to have images for before and after the fire to be able to assess the level of damage.

We also want to create an ROI (region of interest) and zoom in to the area of interest. We do so by applying a buffer around our POI.

In [ ]:
# create a list of images we want (before, after, some years later)
sentinel_sequence = [0, 1, 4]

In [ ]:
# Define a region of interest with a buffer zone of 6 km (5_000 metres)
roi = poi.buffer(5_000)

In [ ]:
# download_url = ee.Image(sentinel_list.get(4)).visualize(
#     **{
#         'min': 0,
#         'max': 3_000,
#         'bands': ['B4', 'B3', 'B2'],
#     }
# ).getDownloadURL(
#     {
#         'dimensions': 2_850,
#         'region': roi,
#         'format': 'GEO_TIFF',
#     }
# )

# response = requests.get(download_url)
# if response.status_code == 200:
#     with open(IMAGES_PATH / "vesuvius_from_above.tiff", 'wb') as file:
#         file.write(response.content)
#     print("Done!")
# else:
#     print("Error")

In [ ]:
parameters = {
    'min': 0,
    'max': 3_000,
    'dimensions': 2_800,
    'bands': ['B4', 'B3', 'B2'],
    'region': roi,
}

In [ ]:
for i in sentinel_sequence:
    
    # when was this image taken?
    unix_time = ee.Image(sentinel_list.get(i)).get('GENERATION_TIME').getInfo() / 1_000
    date_time = datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d %H:%M:%S')

    # cloud cover
    cloud = ee.Image(sentinel_list.get(i)).get('CLOUD_COVERAGE_ASSESSMENT').getInfo()
    
    print(f'Image #{i} | {date_time} | cloud coverage: {np.round(cloud, 1)}%')
    
    display(
        Image(
            url=ee.Image(sentinel_list.get(i)).getThumbUrl(parameters)
        )
    )

## Normalized Difference Vegetation Index (NDVI)

The normalized difference vegetation index (NDVI) is a simple graphical indicator that can be used to analyze remote sensing measurements, often from a space platform, assessing whether or not the target being observed contains live green vegetation.

- [Source: Agricolus](https://www.agricolus.com/en/vegetation-indices-ndvi-ndmi/)

In [ ]:
# ndvi palette: red is low, green is high vegetation
palette = ['red', 'yellow', 'green']

ndvi_parameters = {
    'min': 0.1,
    'max': 0.5,
    'dimensions': 2_800,
    'palette': palette,
    'region': roi,
}

parameters = {
    'min': 0,
    'max': 3_000,
    'dimensions': 2_800,
    'bands': ['B4', 'B3', 'B2'],
    'region': roi,
}

In [ ]:
# for i in sentinel_sequence:

#     # when was this image taken?
#     unix_time = ee.Image(sentinel_list.get(i)).get('GENERATION_TIME').getInfo() / 1_000
#     date_time = datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d %H:%M:%S')

#     print(f'Image #{i} | {date_time}')
    
#     # display the image
#     display(
#         Image(
#             url=ee.Image(sentinel_list.get(i)).normalizedDifference(['B8', 'B4']).getThumbUrl(ndvi_parameters)
#         )
#     )

In [ ]:
folium.Map.add_ee_layer = add_ee_layer
vesuvius_map = folium.Map(location=[lat, lon], zoom_start=13)

# Add a layer for each satellite image of interest (before, during and after)
for i in sentinel_sequence:

    # when was this image taken?
    unix_time = ee.Image(sentinel_list.get(i)).get('GENERATION_TIME').getInfo() / 1_000
    date_time = datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d %H:%M:%S')

    vesuvius_map.add_ee_layer(
        ee.Image(sentinel_list.get(i)).normalizedDifference(['B8', 'B4']), 
        ndvi_parameters,
        name=date_time,
    )
    
# Add a layer control panel to the map
folium.LayerControl(collapsed = False).add_to(vesuvius_map)

# vesuvius_map.save(IMAGES_PATH / "vesuvius_interactive.html")
display(vesuvius_map)

In [ ]:
num_cols = len(sentinel_sequence)
plt.figure(figsize=(14, 20))

for i, image_index in enumerate(sentinel_sequence):
    # NDVI
    image_ndvi_url = ee.Image(
        sentinel_list.get(image_index)
    ).normalizedDifference(
        ['B8', 'B4']
    ).getThumbUrl(
        ndvi_parameters
    )
    image_ndvi_content = requests.get(image_ndvi_url).content
    image_ndvi = PILImage.open(BytesIO(image_ndvi_content))
    
    plt.subplot(1, num_cols, i+1)
    plt.axis('off')
    plt.imshow(np.asarray(image_ndvi))
    plt.title(r'$\it{NDVI}$', fontsize=12)
    
    # RGB
    image_url = ee.Image(
        sentinel_list.get(image_index)
    ).getThumbUrl(
        parameters
    )
    image_content = requests.get(image_url).content
    image = PILImage.open(BytesIO(image_content))
    
    plt.subplot(2, num_cols, i+1)
    plt.axis('off')
    plt.imshow(np.asarray(image))
    
    unix_time = ee.Image(sentinel_list.get(image_index)).get('GENERATION_TIME').getInfo() / 1_000
    date_time = datetime.fromtimestamp(unix_time, tz=pytz.UTC).strftime('%Y-%m-%d, %H:%M')    
    plt.title(f"{date_time} (UTC)\n" + r"$\it{RGB}$", fontsize=12, fontweight="bold")

plt.subplots_adjust(wspace=0.04, hspace=0.04)
plt.savefig(IMAGES_PATH / "rgb_ndvi.png", bbox_inches='tight', dpi=300)
plt.show()